# Extended Numerical Example with repeats
We will repeat \(R)\ times.

Then, we can calculate the average of the prices and std.dev..


In [13]:
# Set the initial variables for the script
import numpy as np
from numpy.linalg import lstsq

# Define parameters
r = 0.06    # Interest rate
K = 1.00    # Strike price
# it is better to work in time-steps than in years
dt = 1/12   # Time-step size in years
nt = 12     # Number of time-steps
T = nt*dt   # Total time to maturity in years

N = 10000      # Number of simulations
R = 100        # Number of repetitions for the Monte Carlo simulation

# For the stock price, here, we will simulate from a Geometric Brownian motion
# We assume a risk-neutral measure
S0 = 1.00   # Initial stock price
sigma = 0.2  # Volatility of the stock
# Generate the stock price paths
np.random.seed(42)  # For reproducibility
# It is possible to simulate from a Geometric Brownian motion without using a loop
# How ever, for clarity, we will use a loop here

price_estimate = np.zeros(R)  # To store the price estimates for each repetition

# loop over all repeats
for repnum in range(R):
    # We need to clear variables in the loop to avoid carrying over values from previous iterations
    dcf = None
    intrinsic = None
    exec_t = None
    payoff = None
    

    S = np.zeros((N, nt + 1))
    S[:, 0] = S0
    
    for i in range(1, nt + 1):
        Z = np.random.normal(0, 1, N)  # Standard normal random variables
        S[:, i] = S[:, i - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    # Check what is going on by printing the average stock price at maturity
    # print(f"Average stock price at t=1 for repetition {r+1}: {np.mean(S[:, 1]):.4f}")
    # print(r)
        
    # We will calculate the exercise/intrinsic value for each path at maturity
    # recall that the option to price is a put option
    intrinsic = np.maximum(K - S[:,-1], 0)

    # We create a payoff vector that will contain the discounted exercise value.
    payoff = np.copy(intrinsic)

    # Now we set the vector exec_t to maturity for each path
    exec_t = nt * np.ones((N,), dtype=int)  # All paths would optimally be exercised at time-step 3 (so far))

    # Now, we can start the backward algorithm at t=nt-1, and loop all the way back to t=1

    # STEP 1	Discount the cash flows to time "t" based on when it's optimal to exercise for each path
    # STEP 2	Identify the ITM paths in order toregress only on ITM paths
    # STEP 3	Build X matrix for regression
    # STEP 4	build y vector
    # STEP 5	regress to get beta
    # STEP 6	Calculate y_hat to approximate the holding value function
    # STEP 7	Update the optimal exercise time for each path

    # We will do one more thing, for illustration purposes, we will keep the beta coefficients for each time-step in a matrix
    betas = np.zeros((3, nt))  # Store beta coefficients for each time-step

    for t_now in range(nt-1, 0, -1):

        # STEP 1
        dcf = np.exp(-r * dt) * payoff  # Discounted cash flow to time t

        # STEP 2
        itm_paths = np.where(S[:, t_now] < K)[0]  # Identify ITM paths

        # STEP 3
        # Build X matrix for regression (using the stock prices at time t_now)
        # We use a constant, the stock price and the square of the stock price
        X = np.column_stack((np.ones(len(itm_paths)), S[itm_paths, t_now], S[itm_paths, t_now]**2))

        # STEP 4
        # Build y vector (the discounted cash flows for ITM paths)
        y = dcf[itm_paths]

        # STEP 5
        # Perform regression to get beta coefficients
        
        beta = lstsq(X, y, rcond=None)[0]
        # Store the beta coefficients for this time-step
        betas[:,t_now] = beta

        # STEP 6
        # Calculate y_hat to approximate the holding value function
        y_hat = X @ beta

        # Update the payoff for exercised paths to be the current intrinsic value
        intrinsic[itm_paths] = np.maximum(K - S[itm_paths, t_now], 0)

        # STEP 7
        # Update the optimal exercise time for each path
        exec_t[itm_paths] = np.where(y_hat < intrinsic[itm_paths], t_now, exec_t[itm_paths])
        payoff = np.where(exec_t == t_now, intrinsic, dcf)

    # Finallym at t=0, we discount the cash flows to time "t" based on when it's optimal to exercise for each path
    # STEP 1
    dcf = np.exp(-r * dt) * payoff

    # STEP 2
    # To get the price estimate, we simply take the average of the discounted cash flows
    price_estimate[repnum] = np.mean(dcf)

# Just show all price estimates for each repetition
# print(price_estimate)

# Print the average price estimate across all repetitions
print(f"Estimated price of the American put option: {np.mean(price_estimate):.4f}")
print(f"Standard deviation of the price estimates: {np.std(price_estimate):.4f}")

Estimated price of the American put option: 0.0574
Standard deviation of the price estimates: 0.0007
